In [ ]:
import json

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
json_content = json.load(open('fine-tune-annotated.json'))

FileNotFoundError: [Errno 2] No such file or directory: 'fine-tune-annotated.json'

In [ ]:
# messages = []
f = open("fine-tuning-messages.jsonl", "a")
for entry in json_content:
  prompt = entry['prompt']
  rubric = entry['rubric']
  essay = entry['essay']
  score = entry['score']

  system_content = f"You are an essay scorer that is grading essays. Keep in mind they should spell correctly and poor spelling should be penalized with a lower score. Given a prompt, rubric and an essay you should return a single integer representing the score of the essay that is within the bounds of the rubric."
  user_content = f"""Here is the rubric: {rubric}, prompt: {prompt}, and essay: {essay}.
    Provide a single integer representing the score of the essay that is within the bounds of the rubric.
  """
  assistant_content = score

  system = {
      "role": "system",
      "content": system_content
  }

  user = {
      "role": "user",
      "content": user_content
  }
  assistant = {
      "role": "assistant",
      "content": assistant_content
  }


  # message = [system, user, assistant]
  message = {
      "messages": [system, user, assistant]
  }
  f.write(json.dumps(message))
  f.write('\n')
f.close()

In [ ]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

client.files.create(
  file=open("fine-tuning-messages.jsonl", "rb"),
  purpose="fine-tune"
)



FileObject(id='file-sCGW9tKdMCwtdRjkwx89PoM9', bytes=1090011, created_at=1716865851, filename='fine-tuning-messages.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-sCGW9tKdMCwtdRjkwx89PoM9",
  model="gpt-3.5-turbo-0125"
)

FineTuningJob(id='ftjob-yuqa7MxhDI1j4RtQLnFap98x', created_at=1716865921, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-C8yV2ACuxzOYfPp3V8KQMvB2', result_files=[], seed=397626080, status='validating_files', trained_tokens=None, training_file='file-sCGW9tKdMCwtdRjkwx89PoM9', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
from openai import OpenAI

system_content = f"You are an essay scorer that is grading essays. Keep in mind they should spell correctly and poor spelling should be penalized with a lower score. Given a prompt, rubric and an essay you should return a single integer representing the score of the essay that is within the bounds of the rubric. Return a json object"


rubric = "Rubric for Patience Essays:\n\n\"3\": \"Tells a story with ideas that are clearly focused on the topic and are thoroughly developed with specific, relevant details. Organization and connections between ideas and/or events are clear and logically sequenced. Command of language, including effective and compelling word choice and varied sentence structure, clearly supports the writer's purpose and audience. Consistent, appropriate use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation for the grade level.\",\n\"2\": \"Tells a story with ideas that are somewhat focused on the topic and are developed with a mix of specific and/or general details. Organization and connections between ideas and/or events are logically sequenced. Adequate command of language, including effective word choice and clear sentences, supports the writer's purpose and audience. Adequate use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation for the grade level.\",\n\"1\": \"Tells a story with ideas that are minimally focused on the topic and developed with limited and/or general details. Organization and connections between ideas and/or events are weak. Limited use of language, including lack of variety in word choice and sentences, may hinder support for the writer's purpose and audience. Limited use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation for the grade level.\",\n\"0\": \"Ideas are not focused on the task and/or are undeveloped. No organization evident. Ineffective use of language for the writer's purpose and audience. Ineffective use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation."
prompt = "Prompt: Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining. Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience."
essay = "Some story about patience happened."

model = "ft:gpt-3.5-turbo-0125:personal::9TgLi0s0"

user_content = f"""Here is the rubric: {rubric}, prompt: {prompt}, and essay: {essay}.
    Provide a single integer representing the score of the essay that is within the bounds of the rubric.
"""
response = client.chat.completions.create(
  model=model,
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": system_content},
    {"role": "user", "content": user_content}
  ]
)
print(response.choices[0].message)


ChatCompletionMessage(content='{\n  "score": 2\n}', role='assistant', function_call=None, tool_calls=None)


###Evaluation

In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import ast
import pandas as pd
client = OpenAI(api_key=OPENAI_API_KEY)

In [57]:
json_eval_data = json.load(open('aggregated_annotations.json'))

gold_scores = []
given_scores = []


# f = open("evaluation.json", "a")

for entry in json_eval_data:
  prompt = entry['prompt']
  rubric = entry['rubric']
  essay = entry['essay']
  gold_score = entry['score']

  system_content = f"You are an essay scorer that is grading essays. Keep in mind they should spell correctly and poor spelling should be penalized with a lower score. Given a prompt, rubric and an essay you should return a single integer representing the score of the essay that is within the bounds of the rubric. Return a json object with the key 'score' only"


  # rubric = "Rubric for Patience Essays:\n\n\"3\": \"Tells a story with ideas that are clearly focused on the topic and are thoroughly developed with specific, relevant details. Organization and connections between ideas and/or events are clear and logically sequenced. Command of language, including effective and compelling word choice and varied sentence structure, clearly supports the writer's purpose and audience. Consistent, appropriate use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation for the grade level.\",\n\"2\": \"Tells a story with ideas that are somewhat focused on the topic and are developed with a mix of specific and/or general details. Organization and connections between ideas and/or events are logically sequenced. Adequate command of language, including effective word choice and clear sentences, supports the writer's purpose and audience. Adequate use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation for the grade level.\",\n\"1\": \"Tells a story with ideas that are minimally focused on the topic and developed with limited and/or general details. Organization and connections between ideas and/or events are weak. Limited use of language, including lack of variety in word choice and sentences, may hinder support for the writer's purpose and audience. Limited use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation for the grade level.\",\n\"0\": \"Ideas are not focused on the task and/or are undeveloped. No organization evident. Ineffective use of language for the writer's purpose and audience. Ineffective use of conventions of Standard English for grammar, usage, spelling, capitalization, and punctuation."
  # prompt = "Prompt: Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining. Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience."
  # essay = "Some story about patience happened."

  model = "ft:gpt-3.5-turbo-0125:personal::9TgLi0s0"

  user_content = f"""Here is the rubric: {rubric}, prompt: {prompt}, and essay: {essay}.
      Provide a single integer representing the score of the essay that is within the bounds of the rubric.
  """
  response = client.chat.completions.create(
    model=model,
    temperature=.1,
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": system_content},
      {"role": "user", "content": user_content}
    ]
  )
  # print(json.loads(response.choices[0].message.content)['score'])
  given_score = json.loads(response.choices[0].message.content)['score']
  json_entry = {
      'prompt': prompt,
      'rubric': rubric,
      'essay': essay,
      'score': gold_score,
      'gptft_score': given_score
  }

  gold_scores.append(gold_score)
  given_scores.append(given_score)
#   f.write(json.dumps(json_entry))
#   f.write(',\n')
# f.close()


In [58]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


gold_scores = [int(item) for item in gold_scores]
given_scores = [int(item) for item in given_scores]

mae = mean_absolute_error(gold_scores, given_scores)
mse = mean_squared_error(gold_scores, given_scores)
# print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

Mean Squared Error: 1.0925925925925926


In [61]:
json_eval_data = json.load(open('aggregated_annotations.json'))


gpt_scores = []
# f = open("evaluation.json", "a")
model = "gpt-3.5-turbo"

for entry in json_eval_data:
  prompt = entry['prompt']
  rubric = entry['rubric']
  essay = entry['essay']
  gold_score = entry['score']

  system_content = f"You are an essay scorer that is grading essays. Keep in mind they should spell correctly and poor spelling should be penalized with a lower score. Given a prompt, rubric and an essay you should return a single integer representing the score of the essay that is within the bounds of the rubric. Return a json object in the format 'score': given_score"



  user_content = f"""Here is the rubric: {rubric}, prompt: {prompt}, and essay: {essay}.
      Provide a single integer representing the score of the essay that is within the bounds of the rubric.
  """
  response = client.chat.completions.create(
    model=model,
    temperature=0.1,
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": system_content},
      {"role": "user", "content": user_content}
    ]
  )
  # print(json.loads(response.choices[0].message.content)['score'])
  given_score = json.loads(response.choices[0].message.content)['score']
  json_entry = {
      'prompt': prompt,
      'rubric': rubric,
      'essay': essay,
      'score': gold_score,
      'gpt_score': given_score
  }

  gpt_scores.append(given_score)

#   f.write(json.dumps(json_entry))
#   f.write(',\n')
# f.close()



In [62]:
gpt_scores = [int(item) for item in gpt_scores]

mae = mean_absolute_error(gold_scores, gpt_scores)
mse = mean_squared_error(gold_scores, gpt_scores)
# print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.8333333333333334
